### **0. Set-up**

In [1]:
# Import libraries and utils
%run '../../utils.ipynb'

In [2]:
# Get api key
load_dotenv()
OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY')

# Set client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
)

In [3]:
# Load dataframe
en_simlex = pd.read_csv("../../../data/dataset/cleaned-en-simlex-999.csv")

# Select subset
en_simlex = en_simlex.iloc[0:333]

# Convert to tuple
tuples_list = list(zip(en_simlex['word1'], en_simlex['word2']))

In [4]:
# Show results
tuples_list

[('old', 'new'),
 ('smart', 'intelligent'),
 ('hard', 'difficult'),
 ('happy', 'cheerful'),
 ('hard', 'easy'),
 ('fast', 'rapid'),
 ('happy', 'glad'),
 ('short', 'long'),
 ('stupid', 'dumb'),
 ('weird', 'strange'),
 ('wide', 'narrow'),
 ('bad', 'awful'),
 ('easy', 'difficult'),
 ('bad', 'terrible'),
 ('hard', 'simple'),
 ('smart', 'dumb'),
 ('insane', 'crazy'),
 ('happy', 'mad'),
 ('large', 'huge'),
 ('hard', 'tough'),
 ('new', 'fresh'),
 ('sharp', 'dull'),
 ('quick', 'rapid'),
 ('dumb', 'foolish'),
 ('wonderful', 'terrific'),
 ('strange', 'odd'),
 ('happy', 'angry'),
 ('narrow', 'broad'),
 ('simple', 'easy'),
 ('old', 'fresh'),
 ('apparent', 'obvious'),
 ('inexpensive', 'cheap'),
 ('nice', 'generous'),
 ('weird', 'normal'),
 ('weird', 'odd'),
 ('bad', 'immoral'),
 ('sad', 'funny'),
 ('wonderful', 'great'),
 ('guilty', 'ashamed'),
 ('beautiful', 'wonderful'),
 ('confident', 'sure'),
 ('dumb', 'dense'),
 ('large', 'big'),
 ('nice', 'cruel'),
 ('impatient', 'anxious'),
 ('big', 'broad'),

### **1. Define and Evaluate Parameters**

In [5]:
prompt = ("Rate the semantic similarity of the word pair: [('{word1}'), ('{word2}')] on a scale from 0 to 10, "
          "where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. "
          "Use two decimals. The response should strictly adhere to the structure: [('word1', 'word2', <score>)]. "
          "Do not provide additional explanations or context.")

In [6]:
# Define model
model = "mistralai/mixtral-8x7b-instruct"

# Set sample size
sample_size = 15

# Delay between individual API calls
delay = 2.0

# Define number of sublists
n_sublists = 333

In [7]:
# Split list
chunks = split_into_n_lists(tuples_list, n_sublists)

# Count number of lists
print(len(chunks))

333


In [8]:
# Print prompts for each chunk
print_prompts_single(chunks, sample_size, prompt)

Rate the semantic similarity of the word pair: [('old'), ('new')] on a scale from 0 to 10, where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. Use two decimals. The response should strictly adhere to the structure: [('word1', 'word2', <score>)]. Do not provide additional explanations or context.
Rate the semantic similarity of the word pair: [('old'), ('new')] on a scale from 0 to 10, where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. Use two decimals. The response should strictly adhere to the structure: [('word1', 'word2', <score>)]. Do not provide additional explanations or context.
Rate the semantic similarity of the word pair: [('old'), ('new')] on a scale from 0 to 10, where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. Use two decimals. The response should strictly adhere to the structure: [('word1', 'word2', <score>)]. Do not provide additional explanations or cont

In [9]:
# Load encoding
encoding = tiktoken.get_encoding("cl100k_base")

# Count tokens per chunk
token_counts = count_tokens_with_tiktoken_single(chunks, prompt)

# Show results
print("Token counts for each formatted prompt:", token_counts)

Token counts for each formatted prompt: [74, 75, 75, 76, 74, 74, 75, 74, 76, 76, 75, 75, 75, 75, 74, 75, 76, 74, 74, 75, 74, 75, 74, 77, 77, 75, 75, 76, 74, 74, 76, 76, 75, 75, 75, 75, 75, 76, 76, 76, 75, 75, 74, 76, 76, 75, 75, 75, 75, 76, 74, 75, 74, 75, 75, 76, 75, 75, 75, 74, 74, 76, 76, 77, 77, 76, 75, 76, 75, 74, 75, 76, 75, 75, 75, 75, 76, 74, 74, 75, 74, 75, 76, 76, 76, 78, 76, 76, 75, 76, 78, 75, 75, 76, 75, 74, 74, 75, 74, 74, 75, 76, 76, 76, 76, 75, 76, 74, 75, 75, 75, 75, 74, 76, 74, 74, 74, 76, 74, 74, 75, 75, 75, 74, 74, 74, 75, 75, 75, 74, 74, 74, 74, 74, 74, 76, 75, 76, 76, 74, 75, 74, 74, 75, 75, 75, 75, 76, 75, 76, 77, 74, 76, 75, 74, 75, 74, 75, 74, 74, 74, 76, 76, 75, 74, 75, 75, 75, 75, 74, 76, 74, 75, 75, 75, 77, 75, 74, 74, 74, 74, 74, 74, 75, 75, 76, 77, 75, 75, 75, 75, 76, 75, 76, 75, 74, 74, 76, 75, 74, 74, 75, 75, 75, 75, 74, 76, 75, 75, 75, 76, 75, 75, 74, 74, 76, 74, 75, 75, 74, 74, 74, 74, 75, 75, 74, 75, 76, 75, 74, 75, 74, 74, 75, 76, 74, 74, 75, 77, 74,

In [10]:
# Max RPD = 10.000
len(token_counts*15)

4995

In [11]:
# Number of total tokens
print(sum(token_counts))

24955


### **2. Extract Data**

In [12]:
# Get results from API
response = get_responses_single(prompt, chunks, model, sample_size, delay)

Processing: 100%|██████████| 4995/4995 [3:24:54<00:00,  2.46s/chunk]  

Total time taken: 12294.18 seconds


In [13]:
# Define filepath
file_path = '../../../data/mixtral-8x7b-instruct/response/en/f9-1.json'

# Ensure directory exists
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Check if file already exists
if not os.path.exists(file_path):
    with open(file_path, 'w') as f:
        json.dump(response, f)
        print("File saved successfully.")
else:
    print("File already exists. JSON was not saved to prevent overwriting.")

File already exists. JSON was not saved to prevent overwriting.


### **3. Process and Inspect Data**

In [14]:
# Process data into dictionary
data_dict = process_responses(response)

# Check for values higher/lower then sample size
higher_lower_samples = {key: value for key, value in data_dict.items() if len(value) < sample_size or len(value) > sample_size}

# Show results
print(higher_lower_samples)

{('apple', 'fruit'): [8.89, 8.87, 8.83], ('happy', 'glad'): [9.2, 9.24, 9.2, 9.24, 9.2, 9.2, 9.2, 9.25, 9.2, 9.2, 9.2, 9.25, 9.25, 9.25], ('short', 'long'): [8.23, 8.2, 8.2, 8.2, 1.0, 8.2, 0.1, 0.1, 1.0, 8.2, 1.0, 8.2, 8.2], ('stupid', 'dumb'): [9.25, 9.25, 9.2, 9.25, 9.23], ('wide', 'narrow'): [1.0, 1.0, 1.0], ('easy', 'difficult'): [0.05, 0.1, 0.1, 0.1, 0.1, 0.1, 0.0, 0.1, 0.1, 0.0, 0.1, 0.1, 0.1], ('bad', 'terrible'): [9.25, 9.25, 9.23, 9.23, 9.23, 9.23, 9.25], ('happy', 'mad'): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], ('large', 'huge'): [9.23, 9.23, 9.23], ('sharp', 'dull'): [2.0, 2.0, 8.13, 8.13, 8.13, 8.13, 8.13, 8.13, 1.75], ('quick', 'rapid'): [9.5, 9.5, 9.5, 9.5, 9.5, 9.5, 9.5, 9.5, 9.5, 9.5], ('dumb', 'foolish'): [8.5, 8.5, 9.0, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5], ('wonderful', 'terrific'): [9.0, 9.11, 9.12, 9.12, 9.12, 9.12, 9.15, 9.12, 9.13, 9.24, 9.11, 9.11, 9.0, 9.0], ('strange', 'odd'): [8.5, 8.5, 9.0, 9.15, 8.5, 8.5, 8.5], ('happy', 'angry'): [0.0, 0.15, 0.15, 0.15, 0.

In [15]:
# Print duplicate word pairs
print_duplicate_word_pairs(en_simlex, data_dict)

        Combined_Columns
45             big_broad
111         wife_husband
126           horse_colt
142        winter_summer
146         roof_ceiling
147    disease_infection
172        flower_violet
176          rod_curtain
192           nail_thumb
197       decade_century
213          grass_blade
217         church_choir
232            water_ice
242            yard_inch
243        molecule_atom
293           water_salt
295  situation_condition
Empty DataFrame
Columns: [Combined_Columns]
Index: []


In [16]:
# Convert dict to Pandas DataFrame
df = create_dataframe(data_dict)

# Show results
df

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
0,old,new,0.00,0.00,0.10,0.00,0.00,0.00,0.00,0.00,0.10,0.00,0.10,0.00,0.00,0.00,0.00
1,smart,intelligent,9.50,9.20,9.50,9.50,9.50,9.50,9.50,9.50,9.00,9.50,9.50,9.50,9.20,9.20,9.20
2,hard,difficult,10.00,9.50,10.00,9.25,9.23,10.00,9.25,9.25,9.20,9.20,9.25,10.00,10.00,10.00,10.00
3,happy,cheerful,9.15,9.15,9.11,9.15,9.15,9.15,9.11,9.25,9.11,9.11,9.15,9.13,9.25,9.11,8.50
4,apple,fruit,8.89,8.87,8.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,heart,surgery,3.45,3.45,2.15,7.23,3.45,3.45,2.15,3.45,3.45,2.15,3.45,3.45,3.45,3.45,2.15
314,woman,secretary,0.63,0.63,3.50,6.25,7.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
315,man,father,7.50,7.50,7.83,7.50,7.50,7.50,7.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
316,beach,island,7.23,7.23,7.23,7.23,7.23,7.23,7.23,7.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Count null values
count_null_values = df.isnull().sum()

# Show results
print("Null value counts per column:", count_null_values)

Null value counts per column: word1                    0
word2                    0
similarity_score_1       0
similarity_score_2       6
similarity_score_3      14
similarity_score_4      23
similarity_score_5      29
similarity_score_6      40
similarity_score_7      51
similarity_score_8      70
similarity_score_9      92
similarity_score_10    109
similarity_score_11    121
similarity_score_12    129
similarity_score_13    135
similarity_score_14    142
similarity_score_15    177
dtype: int64


In [18]:
# Check for rows with at least one null value
rows_with_null = df[df.isnull().any(axis=1)]

# Show results
rows_with_null

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
4,apple,fruit,8.89,8.87,8.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,happy,glad,9.20,9.24,9.20,9.24,9.20,9.20,9.20,9.25,9.2,9.2,9.20,9.25,9.25,9.25,NaN
8,short,long,8.23,8.20,8.20,8.20,1.00,8.20,0.10,0.10,1.0,8.2,1.00,8.20,8.20,NaN,NaN
9,stupid,dumb,9.25,9.25,9.20,9.25,9.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,wide,narrow,1.00,1.00,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,air,helium,7.00,7.00,7.20,7.00,7.20,7.20,7.20,7.00,7.0,7.0,4.25,4.25,7.20,7.20,NaN
312,son,father,9.15,9.15,9.11,9.15,9.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
314,woman,secretary,0.63,0.63,3.50,6.25,7.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
315,man,father,7.50,7.50,7.83,7.50,7.50,7.50,7.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **4. Fix Faulty Word Pairs**

In [ ]:
# ...

### **5. Export Data**

In [ ]:
# Define file_path
file_path = '../../../data/mixtral-8x7b-instruct/processed/en/f9-1.csv'

# Ensure directory exists
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Check if file already exists
if not os.path.exists(file_path):
    df.to_csv(file_path, index=False)
    print("File saved successfully.")
else:
    print("File already exists. Dataframe was not saved to prevent overwriting.")